## [Dynamic Programming](https://harderchoices.com/2018/02/26/dynamic-programming-in-python-reinforcement-learning/)

Dynamic Programming: Collection of methods used to calculate the optimal policies (solve the Bellman equations)

Limitations:

- It needs perfect environment model in the form of Markov Decision Process
- Computationally expensive: solving the Bellman equation is a brute force job.

Why is it important?

Other reinforcement learning algorithms try to do pretty much the same thing except with fewer resources and imperfect environment model.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
from random import randint, random
from time import sleep

In [2]:
# Utitilies:

def print_board(agent_position):
    fields = list(range(16))
    board = "-----------------\n"
    for i in range(0, 16, 4):
        line = fields[i:i+4]
        for field in line:
            if field == agent_position:
                board += "| A "
            elif field == fields[0] or field == fields[-1]:
                board += "| X "
            else:
                board += "|   "
        board += "|\n"
        board += "-----------------\n"     
    print(board)
    
print_board(8)

-----------------
| X |   |   |   |
-----------------
|   |   |   |   |
-----------------
| A |   |   |   |
-----------------
|   |   |   | X |
-----------------



In [3]:
def create_state_to_state_prime_verbose_map():
    l = list(range(16))
    state_to_state_prime = {}
    for i in l:
        if i == 0 or i == 15:
            state_to_state_prime[i] = {'N': 0, 'E': 0, 'S': 0, 'W': 0}
        elif i % 4 == 0:
            state_to_state_prime[i] = {'N': i - 4 if i - 4 in l else i, 'E': i + 1 if i + 1 in l else i, 'S': i + 4 if i + 4 in l else i, 'W': i}
        elif i % 4 == 3:
            state_to_state_prime[i] = {'N': i - 4 if i - 4 in l else i, 'E': i, 'S': i + 4 if i + 4 in l else i, 'W': i - 1 if i - 1 in l else i}
        else:
            state_to_state_prime[i] = {'N': i - 4 if i - 4 in l else i, 'E': i + 1 if i + 1 in l else i, 'S': i + 4 if i + 4 in l else i, 'W': i - 1 if i - 1 in l else i}

    return state_to_state_prime

In [4]:
def create_random_policy():
    return {i: {'N': 0.0, 'E': 0.0, 'S': 0.0, 'W': 0.0} 
            if i == 0 or i == 15 
            else {'N': 0.25, 'E': 0.25, 'S': 0.25, 'W': 0.25} for i in range(16)
           } # [N, E, S, W]

In [5]:
def create_probability_map():
    states = list(range(16))
    state_to_state_prime = create_state_to_state_prime_verbose_map()
    
    probability_map = {}
    
    for state in states:
        for move in ["N", "E", "S", "W"]:
            for prime in states:
                probability_map[(prime, -1, state, move)] = 0 if prime != state_to_state_prime[state][move] else 1
            
    return probability_map

In [6]:
# Agent

def agent(policy, starting_position=None, verbose=False):
    l = list(range(16))
    state_to_state_prime = create_state_to_state_prime_verbose_map()
    agent_position = randint(1, 14) if starting_position is None else starting_position
        
    step_number = 1
    action_taken = None
    
    if verbose:
        print("Move: {} Position: {} Action: {}".format(step_number, agent_position, action_taken))
        print_board(agent_position)
        print("\n")
        sleep(2)
    
    while not (agent_position == 0 or agent_position == 15):
        if verbose:
            clear_output(wait=True)
            print("Move: {} Position: {} Action: {}".format(step_number, agent_position, action_taken))
            print_board(agent_position)
            print("\n")
            sleep(1)
        
        current_policy = policy[agent_position]
        next_move = random()
        lower_bound = 0
        for action, chance in current_policy.items():
            if chance == 0:
                continue
            if lower_bound <= next_move < lower_bound + chance:
                agent_position = state_to_state_prime[agent_position][action]
                action_taken = action
                break 
            lower_bound = lower_bound + chance
                
        step_number += 1   
                
    if verbose:
        clear_output(wait=True)
        print("Move: {} Position: {} Action: {}".format(step_number, agent_position, action_taken))
        print_board(agent_position)
        print("Win!")
    
    return step_number

### Random policy test

In [7]:
data = []

for i in range(1000):
    clear_output(wait=True)
    print("{}%\n".format((i + 1) / 10))
    data.append(agent(create_random_policy()))
    
print("Average steps to finish: {}".format(sum(data)/len(data)))

100.0%

Average steps to finish: 19.359


In [8]:
agent(create_random_policy(), verbose=True)

Move: 7 Position: 15 Action: S
-----------------
| X |   |   |   |
-----------------
|   |   |   |   |
-----------------
|   |   |   |   |
-----------------
|   |   |   | A |
-----------------

Win!


7

### Create greedy policy based on V(s)

In [9]:
def create_greedy_policy(V_s):
    s_to_sprime = create_state_to_state_prime_verbose_map()
    policy = {}
        
    for state in range(16):
        
        state_values = {a: V_s[s_to_sprime[state][a]] for a in ['N', 'S', 'E', 'W']}
        
        if state == 0 or state == 15:
            policy[state] = {'N': 0.0, 'E': 0.0, 'S': 0.0, 'W': 0.0}
        else:
            max_actions = [k for k, v in state_values.items() if v == max(state_values.values())]
            policy[state] = {a: 1 / len(max_actions) if a in max_actions else 0.0 for a in ['N', 'S', 'E', 'W']}
    return policy

Think about Finite-MDP:

p(s' r| s, a)

The set is exhaustive.... contains all possibilities even those not allowed by the game.

In [24]:
def iterative_policy_evaluation(policy, theta=0.01, discount_rate=0.5):
    V_s = {i: 0 for i in range(16)} # 1.
    probablitiy_map = create_probability_map() # 2.

    delta = 100 # 3.
    while not delta < theta: # 4.
        delta = 0 # 5.
        for state in range(16): # 6.
            v = V_s[state] # 7.
            
            total = 0 # 8.
            for action in ["N", "E", "S", "W"]:
                action_total = 0
                for state_prime in range(16):
                    action_total += probablitiy_map[(state_prime, -1, state, action)] * (-1 + discount_rate * V_s[state_prime])
                total += policy[state][action] * action_total   
                
            V_s[state] = round(total, 1) # 9.
            delta = max(delta, abs(v - V_s[state])) # 10.
    return V_s # 11.

In [25]:
from pprint import pprint

In [27]:
policy = create_random_policy()
V_s = iterative_policy_evaluation(policy)
policy = create_greedy_policy(V_s)
print(V_s)

V_s = iterative_policy_evaluation(policy)
policy = create_greedy_policy(V_s)
print(V_s)

{0: 0.0, 1: -1.7, 2: -1.9, 3: -1.9, 4: -1.7, 5: -1.9, 6: -1.9, 7: -1.9, 8: -1.9, 9: -1.9, 10: -1.9, 11: -1.7, 12: -1.9, 13: -1.9, 14: -1.7, 15: 0.0}
{0: 0.0, 1: -1.0, 2: -1.5, 3: -1.8, 4: -1.0, 5: -1.5, 6: -1.8, 7: -1.5, 8: -1.5, 9: -1.8, 10: -1.5, 11: -1.0, 12: -1.8, 13: -1.5, 14: -1.0, 15: 0.0}


In [33]:
data = []

for i in range(1000):
    clear_output(wait=True)
    print("{}%\n".format((i + 1) / 10))
    data.append(agent(policy))
    
print("Average steps to finish: {}".format(sum(data)/len(data)))

100.0%

Average steps to finish: 2.968


In [36]:
agent(policy, verbose=True)

Move: 3 Position: 15 Action: E
-----------------
| X |   |   |   |
-----------------
|   |   |   |   |
-----------------
|   |   |   |   |
-----------------
|   |   |   | A |
-----------------

Win!


3

theta: controls the degree of approximation (smaller = more precise)
discount_rate: dimishes the reward received in future

### Value Iteration

Policy improvement (greedy) + policy evaluation takes too much time.

Use just values.

In [29]:
def value_iteration(V_s, theta=0.01, discount_rate=0.5):
    probablitiy_map = create_probability_map()

    delta = 100
    while not delta < theta:
        delta = 0
        for state in range(1, 15):
            v = V_s[state]
            
            totals = {}
            for action in ["N", "S", "E", "W"]:
                total = 0
                for state_prime in range(16):
                    total += probablitiy_map[(state_prime, -1, state, action)] * (-1 + discount_rate * V_s[state_prime])
                totals[action] = total
            
            V_s[state] = round(max(totals.values()), 4)
            delta = max(delta, abs(v - V_s[state]))
    return V_s

In [30]:
V_s = {i: 0 for i in range(16)}
V_s = value_iteration(V_s)
print(V_s)
V_s = value_iteration(V_s)
print(V_s)

policy = create_greedy_policy(V_s)

{0: 0, 1: -1.0, 2: -1.5, 3: -1.75, 4: -1.0, 5: -1.5, 6: -1.75, 7: -1.5, 8: -1.5, 9: -1.75, 10: -1.5, 11: -1.0, 12: -1.75, 13: -1.5, 14: -1.0, 15: 0}
{0: 0, 1: -1.0, 2: -1.5, 3: -1.75, 4: -1.0, 5: -1.5, 6: -1.75, 7: -1.5, 8: -1.5, 9: -1.75, 10: -1.5, 11: -1.0, 12: -1.75, 13: -1.5, 14: -1.0, 15: 0}


In [22]:
data = []

for i in range(1000):
    clear_output(wait=True)
    print("{}%\n".format((i + 1) / 10))
    data.append(agent(policy))
    
print("Average steps to finish: {}".format(sum(data)/len(data)))

100.0%

Average steps to finish: 3.026


In [23]:
agent(policy, verbose=True)

Move: 3 Position: 0 Action: W
-----------------
| A |   |   |   |
-----------------
|   |   |   |   |
-----------------
|   |   |   |   |
-----------------
|   |   |   | X |
-----------------

Win!


3

In [25]:
create_state_to_state_prime_verbose_map()

{0: {'E': 0, 'N': 0, 'S': 0, 'W': 0},
 1: {'E': 2, 'N': 1, 'S': 5, 'W': 0},
 2: {'E': 3, 'N': 2, 'S': 6, 'W': 1},
 3: {'E': 3, 'N': 3, 'S': 7, 'W': 2},
 4: {'E': 5, 'N': 0, 'S': 8, 'W': 4},
 5: {'E': 6, 'N': 1, 'S': 9, 'W': 4},
 6: {'E': 7, 'N': 2, 'S': 10, 'W': 5},
 7: {'E': 7, 'N': 3, 'S': 11, 'W': 6},
 8: {'E': 9, 'N': 4, 'S': 12, 'W': 8},
 9: {'E': 10, 'N': 5, 'S': 13, 'W': 8},
 10: {'E': 11, 'N': 6, 'S': 14, 'W': 9},
 11: {'E': 11, 'N': 7, 'S': 15, 'W': 10},
 12: {'E': 13, 'N': 8, 'S': 12, 'W': 12},
 13: {'E': 14, 'N': 9, 'S': 13, 'W': 12},
 14: {'E': 15, 'N': 10, 'S': 14, 'W': 13},
 15: {'E': 0, 'N': 0, 'S': 0, 'W': 0}}

In [26]:
create_random_policy()

{0: {'E': 0.0, 'N': 0.0, 'S': 0.0, 'W': 0.0},
 1: {'E': 0.25, 'N': 0.25, 'S': 0.25, 'W': 0.25},
 2: {'E': 0.25, 'N': 0.25, 'S': 0.25, 'W': 0.25},
 3: {'E': 0.25, 'N': 0.25, 'S': 0.25, 'W': 0.25},
 4: {'E': 0.25, 'N': 0.25, 'S': 0.25, 'W': 0.25},
 5: {'E': 0.25, 'N': 0.25, 'S': 0.25, 'W': 0.25},
 6: {'E': 0.25, 'N': 0.25, 'S': 0.25, 'W': 0.25},
 7: {'E': 0.25, 'N': 0.25, 'S': 0.25, 'W': 0.25},
 8: {'E': 0.25, 'N': 0.25, 'S': 0.25, 'W': 0.25},
 9: {'E': 0.25, 'N': 0.25, 'S': 0.25, 'W': 0.25},
 10: {'E': 0.25, 'N': 0.25, 'S': 0.25, 'W': 0.25},
 11: {'E': 0.25, 'N': 0.25, 'S': 0.25, 'W': 0.25},
 12: {'E': 0.25, 'N': 0.25, 'S': 0.25, 'W': 0.25},
 13: {'E': 0.25, 'N': 0.25, 'S': 0.25, 'W': 0.25},
 14: {'E': 0.25, 'N': 0.25, 'S': 0.25, 'W': 0.25},
 15: {'E': 0.0, 'N': 0.0, 'S': 0.0, 'W': 0.0}}

In [27]:
len(create_probability_map())

1024

In [28]:
V_s

{0: 0,
 1: -1.0,
 2: -1.5,
 3: -1.75,
 4: -1.0,
 5: -1.5,
 6: -1.75,
 7: -1.5,
 8: -1.5,
 9: -1.75,
 10: -1.5,
 11: -1.0,
 12: -1.75,
 13: -1.5,
 14: -1.0,
 15: 0}

In [29]:
policy

{0: {'E': 0.0, 'N': 0.0, 'S': 0.0, 'W': 0.0},
 1: {'E': 0.0, 'N': 0.0, 'S': 0.0, 'W': 1.0},
 2: {'E': 0.0, 'N': 0.0, 'S': 0.0, 'W': 1.0},
 3: {'E': 0.0, 'N': 0.0, 'S': 0.5, 'W': 0.5},
 4: {'E': 0.0, 'N': 1.0, 'S': 0.0, 'W': 0.0},
 5: {'E': 0.0, 'N': 0.5, 'S': 0.0, 'W': 0.5},
 6: {'E': 0.25, 'N': 0.25, 'S': 0.25, 'W': 0.25},
 7: {'E': 0.0, 'N': 0.0, 'S': 1.0, 'W': 0.0},
 8: {'E': 0.0, 'N': 1.0, 'S': 0.0, 'W': 0.0},
 9: {'E': 0.25, 'N': 0.25, 'S': 0.25, 'W': 0.25},
 10: {'E': 0.5, 'N': 0.0, 'S': 0.5, 'W': 0.0},
 11: {'E': 0.0, 'N': 0.0, 'S': 1.0, 'W': 0.0},
 12: {'E': 0.5, 'N': 0.5, 'S': 0.0, 'W': 0.0},
 13: {'E': 1.0, 'N': 0.0, 'S': 0.0, 'W': 0.0},
 14: {'E': 1.0, 'N': 0.0, 'S': 0.0, 'W': 0.0},
 15: {'E': 0.0, 'N': 0.0, 'S': 0.0, 'W': 0.0}}

Major Drawback of DP: Has to iterate through entire state set in MDP (sweep of the state set).

Asynchronous dynamic Programming.